This is the notebook I used for the initial cleaning of my data. I look to understand each column, it's meaning and it's relevance to my goals. I do not drop any columns until I get deeper into my analysis and deem it necessary.

The original data source only included the following context *"This Online Retail II data set contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.The company mainly sells unique all-occasion gift-ware. Many customers of the company are wholesalers."*

The questions I am interested in answering are: 
- what is the average customer lifetime and customer lifetime value for this shop based on the information available?
- is there a model that can accurately forecast the customer lifetime value based on the purchasing habits of a customer during their first 12 months?

In [1]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import getpass

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix, f1_score

In [2]:
df = pd.read_csv('online_retail_II.csv')
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,12/9/2010 20:01,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,12/9/2010 20:01,3.75,17530.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.1+ MB


In [4]:
"""anything without a customer id is not helpful, since we want to look at CLV, so we'll drop any rows without that info.
Product descriptions, while useful, aren't critical to my main goal, so I will fill the nans with a filler text"""

df = df.dropna(subset= 'Customer ID')

df['Description'] = df['Description'].fillna('Unknown Items')

df.isna().sum()

C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\1510702497.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Description'].fillna('Unknown Items')


Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [5]:
"""converting to datetime makes the column more useful for exploration. 
I also created a column with just month and year which will be more helpful to my data manipulation later"""

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year

df['Customer ID'] = df['Customer ID'].astype(int)

df.info()

C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\294182057.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  int32         
 7   Country      417534 non-null  object        
 8   Month        417534 non-null  int32         
 9   Year         417534 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(1), object(4)
memory usage: 30.3+ MB


C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\294182057.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['InvoiceDate'].dt.month
C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\294182057.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['InvoiceDate'].dt.year
C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\294182057.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  int32         
 7   Country      417534 non-null  object        
 8   Month        417534 non-null  int32         
 9   Year         417534 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(1), object(4)
memory usage: 30.3+ MB


In [7]:
#renaming and standardizing column names
df.rename(columns={'Description':'desc', 'Customer ID':'cust_id', 'InvoiceDate':'date'}, inplace=True)

df.columns = df.columns.str.lower()

# adding a column to see the price multiplied by quantity for each transaction
df['total_price'] = df['quantity'] * df['price']

C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\2166536363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Description':'desc', 'Customer ID':'cust_id', 'InvoiceDate':'date'}, inplace=True)
C:\Users\NessaJay\AppData\Local\Temp\ipykernel_27152\2166536363.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_price'] = df['quantity'] * df['price']


In [10]:
"""creating data frames based on the type of transaction"""

returns = df[df["total_price"] < 0]
returns

sales = df[df["total_price"] >= 0]
sales

In [15]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407695 entries, 0 to 525460
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      407695 non-null  object        
 1   stockcode    407695 non-null  object        
 2   desc         407695 non-null  object        
 3   quantity     407695 non-null  int64         
 4   date         407695 non-null  datetime64[ns]
 5   price        407695 non-null  float64       
 6   cust_id      407695 non-null  int32         
 7   country      407695 non-null  object        
 8   month        407695 non-null  int32         
 9   year         407695 non-null  int32         
 10  total_price  407695 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(3), int64(1), object(4)
memory usage: 32.7+ MB


In [13]:
#now that the data frame is clean i want to save it as a csv I can use in other notebooks.

df.to_csv('data_clean.csv', index=False)

In [16]:
sales.to_csv('sales_data_clean.csv', index=False)
returns.to_csv('returns_data_clean.csv', index=False)

In [ ]:
"""I decided to send the tables to SQL workbench in case I want to use that tool later on."""


password = getpass.getpass("Please enter the SQL database password: ")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'
engine = create_engine(connection_string)


with engine.connect() as connection:
    query = text('CREATE DATABASE retail;')
    result = connection.execute(query)

result

sales.to_sql(name='sales', con=engine, if_exists='replace', index=False, schema= 'retail')
returns.to_sql(name='returns', con=engine, if_exists='replace', index=False, schema= 'retail')